In [20]:
import numpy as np
import pandas as pd

In [21]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
playlistDF_test = pd.read_csv('C:\\Users\\Akhil_S_Bhat\\Documents\\AIML\\Machine_Learning\\Unsupervised Learning\\Data\\3_Spotify_Data\\spotify_test_data.csv')

In [24]:
complete_feature_set = pd.read_csv('C:\\Users\\Akhil_S_Bhat\\Documents\\AIML\\Machine_Learning\\Unsupervised Learning\\Data\\3_Spotify_Data\\complete_feature.csv')

In [25]:
complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlistDF_test['id'].values)]

In [26]:
complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlistDF_test['id'].values)]

In [27]:
complete_feature_set_playlist_vector = complete_feature_set_playlist.drop(columns=['Unnamed: 0','id','artist_name','track_name','artist_uri','album_uri','album_name','genres']).sum(axis=0)

In [28]:
client_credentials_manager = SpotifyClientCredentials(client_id='cf5e39da3b50438493846ee960df886d',client_secret='23e7cc1dcb4f4836bc594667c6e2cef3')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist = sp.playlist('5j14kA383XcoO33Px4HFPL')

playlist_df = pd.DataFrame(columns=['artist_name','track_uri','artist_uri','track_name','album_uri','duration_ms','album_name'])

for i in range(0,len(playlist['tracks']['items'])):
    attribute_list=[]
    attribute_list.append(playlist['tracks']['items'][i]['track']['artists'][0]['name'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['uri'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['artists'][0]['uri'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['name'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['album']['uri'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['duration_ms'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['album']['name'])
    playlist_df.loc[len(playlist_df)] = attribute_list

playlist_df['track_uri'] = playlist_df['track_uri'].apply(lambda x:x[x.rfind(':')+1:])

playlist_df.head()

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,Creedence Clearwater Revival,2LawezPeJhN4AWuSB0GtAU,spotify:artist:3IYUhFvPQItj6xySrBmZkd,Have You Ever Seen The Rain,spotify:album:372cMadhAGlNuDnc8TssqF,160133,Pendulum (40th Anniversary Edition)
1,Queen,4rDbp1vnvEhieiccprPMdI,spotify:artist:1dfeR4HaWDbWqFHLkxsg1d,Somebody To Love - Remastered 2011,spotify:album:4KZGe18wZJbXL6JLW4KyLc,296480,A Day At The Races (Deluxe Edition 2011 Remaster)
2,TOTO,2374M0fQpWi3dLnB54qaLX,spotify:artist:0PFtn5NtBbbUNbU9EAmIWF,Africa,spotify:album:62U7xIHcID94o20Of5ea4D,295893,Toto IV
3,Boston,1QEEqeFIZktqIpPI4jSVSF,spotify:artist:29kkCKKGXheHuoO829FxWK,More Than a Feeling,spotify:album:2QLp07RO6anZHmtcKTEvSC,285133,Boston
4,Bon Jovi,37ZJ0p5Jm13JPevGcx4SkF,spotify:artist:58lV9VcRSjABbAbfWS6skp,Livin' On A Prayer,spotify:album:0kBfgEilUFCMIQY5IOjG4t,249293,Slippery When Wet


In [29]:
playlist_df.shape

(92, 7)

In [30]:
non_playlist_df = playlist_df[playlist_df['track_uri'].isin(complete_feature_set_nonplaylist['id'].values)]

In [31]:
len(complete_feature_set_playlist_vector.values.reshape(1, -1))

1

In [32]:
complete_feature_set_nonplaylist = complete_feature_set_nonplaylist.drop(columns=['Unnamed: 0','id','artist_name','track_name','artist_uri','album_uri','album_name','genres'],axis=1)

In [33]:
non_playlist_df = non_playlist_df.drop_duplicates()

In [34]:
non_playlist_df.shape

(41, 7)

In [35]:
non_playlist_df['sim'] = cosine_similarity(complete_feature_set_nonplaylist.values, complete_feature_set_playlist_vector.values.reshape(1, -1))[:,0]

In [36]:
non_playlist_df_top_10 = non_playlist_df.sort_values('sim',ascending = False).head(10)

In [37]:
non_playlist_df_top_10[['artist_name','track_name']]

,artist_name,track_name
49,Neil Young,Rockin' in the Free World
75,Led Zeppelin,Whole Lotta Love - 1990 Remaster
33,Queen,Bohemian Rhapsody - Remastered 2011
65,Journey,Separate Ways (Worlds Apart)
19,Queen,Somebody To Love - Remastered 2011
46,Pink Floyd,Hey You
1,Queen,Somebody To Love - Remastered 2011
58,Scorpions,Rock You Like a Hurricane
22,U2,Sunday Bloody Sunday - Remastered 2008
9,Alice Cooper,Poison
